In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [1]:
from keras.layers import BatchNormalization
from sklearn.metrics import mean_squared_error
from keras.initializers import RandomNormal
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD,Adam
from keras.models import Model,Sequential
from keras.layers import *
from keras import backend as K
from keras.models import model_from_json
from matplotlib import cm as CM
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import scipy.io as io
from PIL import Image
import PIL
import h5py
import os
import glob
import cv2
import random
import math
import sys
import numpy as np


In [2]:
from keras.utils import load_img, img_to_array

In [3]:
K.clear_session()
root = os.path.join(os.getcwd(),'dataset')
print(root)

C:\Users\User\CROWD-COUNTING-USING-CSRNET-master\dataset


In [4]:
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
temp = 'test_images'
path_sets = [part_B_train]
print(part_B_train)

C:\Users\User\CROWD-COUNTING-USING-CSRNET-master\dataset\part_B_final/train_data\images


In [5]:
img_paths = []

for path in path_sets:
    
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        
        img_paths.append(str(img_path))
        
print("Total images : ",len(img_paths))


Total images :  400


In [6]:
def create_img(path):
    #Function to load,normalize and return image 
    im = Image.open(path).convert('RGB')
    
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225

    print(im.shape)
    #im = np.expand_dims(im,axis  = 0)
    return im

def get_input(path):
    path = path[0] 
    img = create_img(path)
    return(img)
    
    
    
def get_output(path):
    #import target
    #resize target
    
    gt_file = h5py.File(path,'r')
    
    target = np.asarray(gt_file['density'])
    
    img = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64
    
    #img = np.expand_dims(img,axis  = 0)
    
    print(img.shape)
    
    return img
    
    
    
def preprocess_input(image,target):
    #crop image
    #crop target
    #resize target
    crop_size = (int(image.shape[0]/2),int(image.shape[1]/2))
    
    
    if random.randint(0,9)<= -1:            
            dx = int(random.randint(0,1)*image.shape[0]*1./2)
            dy = int(random.randint(0,1)*image.shape[1]*1./2)
    else:
            dx = int(random.random()*image.shape[0]*1./2)
            dy = int(random.random()*image.shape[1]*1./2)

   
    img = image[dx : crop_size[0]+dx , dy:crop_size[1]+dy]
    
    target_aug = target[dx:crop_size[0]+dx,dy:crop_size[1]+dy]
    print("shape",img.shape)

    return(img,target_aug)
    

In [8]:


#Image data generator 
def image_generator(files, batch_size = 64):
    
    while True:
        
        input_path = np.random.choice(a = files, size = batch_size)
        
        batch_input = []
        batch_output = [] 
          
    
        
        inputt = get_input(input_path )
        output = get_output(input_path[0].replace('.jpg','.h5').replace('images','ground_truth') )
            
       
        batch_input += [inputt]
        batch_output += [output]
    

        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        
        yield( batch_x, batch_y )


In [9]:

def save_mod(model , str1 , str2):
    model.save_weights(str1)
    model_json = model.to_json()
    with open(str2, "w") as json_file:
        json_file.write(model_json)


In [10]:
print(os.getcwd())

C:\Users\User\CROWD-COUNTING-USING-CSRNET-master


In [11]:
def init_weights_vgg(model):
    vgg =  VGG16(weights='imagenet', include_top=False)
    
    vgg_weights=[]
    for layer in vgg.layers:
        if('conv' in layer.name):
            vgg_weights.append(layer.get_weights())
    
    
    offset=0
    i=0
    while(i<10):
        if('conv' in model.layers[i+offset].name):
            model.layers[i+offset].set_weights(vgg_weights[i])
            i=i+1
            #print('h')
            
        else:
            offset=offset+1

    return (model)
    



In [13]:


def euclidean_distance_loss(y_true, y_pred):
    # Euclidean distance as a measure of loss (Loss function) 
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [15]:
# Neural network model : VGG + Conv
def CrowdNet():  
            #Variable Input Size
            rows = None
            cols = None
            
            #Batch Normalisation option
            
            batch_norm = 0
            kernel = (3, 3)
            init = RandomNormal(stddev=0.01)
            model = Sequential() 
            
            #custom VGG:
            
            if(batch_norm):
                model.add(Conv2D(64, kernel_size = kernel, input_shape = (rows,cols,3),activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                
            else:
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same',input_shape = (rows, cols, 3), kernel_initializer = init))
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                
                

                
            #Conv2D
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(256, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(128, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(64, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(1, (1, 1), activation='relu', dilation_rate = 1, kernel_initializer = init, padding = 'same'))
        
            sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
            model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['mse'])
            
            model = init_weights_vgg(model)
            
            return model

In [16]:
model = CrowdNet()

C:\Users\User\anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, None, None, 64)    1792      
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 64)   0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 128)   73856     
                                                                 
 conv2d_3 (Conv2D)           (None, None, None, 128)   147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, None, None, 128)  0         
 2D)                                                    

In [18]:
train_gen = image_generator(img_paths, 1)


In [20]:
sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['accuracy', 'mse'])

In [21]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('my_model_weight.h5', save_best_only=False, verbose=1)

In [22]:
model.load_weights('my_model_weight.h5')

In [23]:
model.fit(train_gen, epochs=15, steps_per_epoch=700, verbose=1,initial_epoch = 11, callbacks=[checkpoint])

(768, 1024, 3)
(96, 128)
Epoch 12/15
(768, 1024, 3)
(96, 128)
  1/700 [..............................] - ETA: 3:23:58 - loss: 0.3183 - accuracy: 0.0234 - mse: 0.0047(768, 1024, 3)
(96, 128)
  2/700 [..............................] - ETA: 2:42:17 - loss: 0.2030 - accuracy: 0.0437 - mse: 0.0024(768, 1024, 3)
(96, 128)
  3/700 [..............................] - ETA: 2:39:36 - loss: 0.2578 - accuracy: 0.0422 - mse: 0.0027(768, 1024, 3)
(96, 128)
  4/700 [..............................] - ETA: 2:36:51 - loss: 0.2129 - accuracy: 0.0543 - mse: 0.0021(768, 1024, 3)
(96, 128)
  5/700 [..............................] - ETA: 2:36:58 - loss: 0.2028 - accuracy: 0.0465 - mse: 0.0018(768, 1024, 3)
(96, 128)
  6/700 [..............................] - ETA: 2:34:50 - loss: 0.1855 - accuracy: 0.0390 - mse: 0.0016(768, 1024, 3)
(96, 128)
  7/700 [..............................] - ETA: 2:33:46 - loss: 0.1926 - accuracy: 0.0484 - mse: 0.0016(768, 1024, 3)
(96, 128)
  8/700 [..............................] -

 64/700 [=>............................] - ETA: 2:08:14 - loss: 0.2258 - accuracy: 0.1535 - mse: 0.0036(768, 1024, 3)
(96, 128)
 65/700 [=>............................] - ETA: 2:08:02 - loss: 0.2234 - accuracy: 0.1589 - mse: 0.0036(768, 1024, 3)
(96, 128)
 66/700 [=>............................] - ETA: 2:07:55 - loss: 0.2225 - accuracy: 0.1662 - mse: 0.0035(768, 1024, 3)
(96, 128)
 67/700 [=>............................] - ETA: 2:07:43 - loss: 0.2225 - accuracy: 0.1652 - mse: 0.0035(768, 1024, 3)
(96, 128)
 68/700 [=>............................] - ETA: 2:07:31 - loss: 0.2202 - accuracy: 0.1702 - mse: 0.0035(768, 1024, 3)
(96, 128)
 69/700 [=>............................] - ETA: 2:07:19 - loss: 0.2232 - accuracy: 0.1681 - mse: 0.0035(768, 1024, 3)
(96, 128)
 70/700 [==>...........................] - ETA: 2:07:27 - loss: 0.2219 - accuracy: 0.1672 - mse: 0.0035(768, 1024, 3)
(96, 128)
 71/700 [==>...........................] - ETA: 2:07:17 - loss: 0.2303 - accuracy: 0.1666 - mse: 0.0039(

128/700 [====>.........................] - ETA: 1:53:53 - loss: 0.2184 - accuracy: 0.1871 - mse: 0.0038(768, 1024, 3)
(96, 128)
129/700 [====>.........................] - ETA: 1:53:23 - loss: 0.2173 - accuracy: 0.1871 - mse: 0.0038(768, 1024, 3)
(96, 128)
130/700 [====>.........................] - ETA: 1:52:59 - loss: 0.2165 - accuracy: 0.1872 - mse: 0.0038(768, 1024, 3)
(96, 128)
131/700 [====>.........................] - ETA: 1:52:54 - loss: 0.2175 - accuracy: 0.1864 - mse: 0.0038(768, 1024, 3)
(96, 128)
132/700 [====>.........................] - ETA: 1:52:33 - loss: 0.2169 - accuracy: 0.1884 - mse: 0.0038(768, 1024, 3)
(96, 128)
133/700 [====>.........................] - ETA: 1:52:27 - loss: 0.2191 - accuracy: 0.1893 - mse: 0.0039(768, 1024, 3)
(96, 128)
134/700 [====>.........................] - ETA: 1:52:18 - loss: 0.2178 - accuracy: 0.1880 - mse: 0.0038(768, 1024, 3)
(96, 128)
135/700 [====>.........................] - ETA: 1:52:07 - loss: 0.2212 - accuracy: 0.1875 - mse: 0.0040(

192/700 [=======>......................] - ETA: 1:41:09 - loss: 0.2288 - accuracy: 0.1908 - mse: 0.0042(768, 1024, 3)
(96, 128)
193/700 [=======>......................] - ETA: 1:41:03 - loss: 0.2295 - accuracy: 0.1911 - mse: 0.0042(768, 1024, 3)
(96, 128)
194/700 [=======>......................] - ETA: 1:40:50 - loss: 0.2286 - accuracy: 0.1902 - mse: 0.0042(768, 1024, 3)
(96, 128)
195/700 [=======>......................] - ETA: 1:40:40 - loss: 0.2293 - accuracy: 0.1908 - mse: 0.0042(768, 1024, 3)
(96, 128)
196/700 [=======>......................] - ETA: 1:40:32 - loss: 0.2287 - accuracy: 0.1926 - mse: 0.0042(768, 1024, 3)
(96, 128)
197/700 [=======>......................] - ETA: 1:40:19 - loss: 0.2280 - accuracy: 0.1925 - mse: 0.0042(768, 1024, 3)
(96, 128)
198/700 [=======>......................] - ETA: 1:40:08 - loss: 0.2270 - accuracy: 0.1931 - mse: 0.0042(768, 1024, 3)
(96, 128)
199/700 [=======>......................] - ETA: 1:39:46 - loss: 0.2266 - accuracy: 0.1931 - mse: 0.0041(

256/700 [=========>....................] - ETA: 1:26:34 - loss: 0.2355 - accuracy: 0.1886 - mse: 0.0047(768, 1024, 3)
(96, 128)
257/700 [==========>...................] - ETA: 1:26:21 - loss: 0.2349 - accuracy: 0.1885 - mse: 0.0047(768, 1024, 3)
(96, 128)
258/700 [==========>...................] - ETA: 1:26:06 - loss: 0.2341 - accuracy: 0.1883 - mse: 0.0047(768, 1024, 3)
(96, 128)
259/700 [==========>...................] - ETA: 1:25:53 - loss: 0.2334 - accuracy: 0.1876 - mse: 0.0047(768, 1024, 3)
(96, 128)
260/700 [==========>...................] - ETA: 1:25:37 - loss: 0.2352 - accuracy: 0.1876 - mse: 0.0047(768, 1024, 3)
(96, 128)
261/700 [==========>...................] - ETA: 1:25:24 - loss: 0.2349 - accuracy: 0.1876 - mse: 0.0047(768, 1024, 3)
(96, 128)
262/700 [==========>...................] - ETA: 1:25:11 - loss: 0.2345 - accuracy: 0.1869 - mse: 0.0047(768, 1024, 3)
(96, 128)
263/700 [==========>...................] - ETA: 1:24:58 - loss: 0.2348 - accuracy: 0.1876 - mse: 0.0047(

...

408/700 [================>.............] - ETA: 37:38 - loss: 0.2367 - accuracy: 0.1821 - mse: 0.0048(768, 1024, 3)
(96, 128)
409/700 [================>.............] - ETA: 37:29 - loss: 0.2363 - accuracy: 0.1819 - mse: 0.0048(768, 1024, 3)
(96, 128)
410/700 [================>.............] - ETA: 37:21 - loss: 0.2359 - accuracy: 0.1822 - mse: 0.0047(768, 1024, 3)
(96, 128)
411/700 [================>.............] - ETA: 37:13 - loss: 0.2356 - accuracy: 0.1827 - mse: 0.0047(768, 1024, 3)
(96, 128)
412/700 [================>.............] - ETA: 37:05 - loss: 0.2351 - accuracy: 0.1827 - mse: 0.0047(768, 1024, 3)
(96, 128)
413/700 [================>.............] - ETA: 36:57 - loss: 0.2362 - accuracy: 0.1828 - mse: 0.0048(768, 1024, 3)
(96, 128)
414/700 [================>.............] - ETA: 36:48 - loss: 0.2373 - accuracy: 0.1829 - mse: 0.0048(768, 1024, 3)
(96, 128)
415/700 [================>.............] - ETA: 36:40 - loss: 0.2371 - accuracy: 0.1828 - mse: 0.0048(768, 1024, 3)
(9

473/700 [===================>..........] - ETA: 28:54 - loss: 0.2354 - accuracy: 0.1824 - mse: 0.0048(768, 1024, 3)
(96, 128)
474/700 [===================>..........] - ETA: 28:46 - loss: 0.2352 - accuracy: 0.1822 - mse: 0.0048(768, 1024, 3)
(96, 128)
475/700 [===================>..........] - ETA: 28:38 - loss: 0.2350 - accuracy: 0.1825 - mse: 0.0048(768, 1024, 3)
(96, 128)
476/700 [===================>..........] - ETA: 28:30 - loss: 0.2353 - accuracy: 0.1824 - mse: 0.0048(768, 1024, 3)
(96, 128)
477/700 [===================>..........] - ETA: 28:22 - loss: 0.2349 - accuracy: 0.1820 - mse: 0.0048(768, 1024, 3)
(96, 128)
478/700 [===================>..........] - ETA: 28:14 - loss: 0.2348 - accuracy: 0.1823 - mse: 0.0047(768, 1024, 3)
(96, 128)
479/700 [===================>..........] - ETA: 28:06 - loss: 0.2350 - accuracy: 0.1824 - mse: 0.0047(768, 1024, 3)
(96, 128)
480/700 [===================>..........] - ETA: 27:58 - loss: 0.2364 - accuracy: 0.1821 - mse: 0.0048(768, 1024, 3)
(9

538/700 [======================>.......] - ETA: 20:26 - loss: 0.2337 - accuracy: 0.1840 - mse: 0.0047(768, 1024, 3)
(96, 128)
539/700 [======================>.......] - ETA: 20:18 - loss: 0.2338 - accuracy: 0.1839 - mse: 0.0047(768, 1024, 3)
(96, 128)
540/700 [======================>.......] - ETA: 20:11 - loss: 0.2336 - accuracy: 0.1835 - mse: 0.0047(768, 1024, 3)
(96, 128)
541/700 [======================>.......] - ETA: 20:03 - loss: 0.2338 - accuracy: 0.1837 - mse: 0.0047(768, 1024, 3)
(96, 128)
542/700 [======================>.......] - ETA: 19:55 - loss: 0.2337 - accuracy: 0.1834 - mse: 0.0047(768, 1024, 3)
(96, 128)
543/700 [======================>.......] - ETA: 19:47 - loss: 0.2337 - accuracy: 0.1833 - mse: 0.0046(768, 1024, 3)
(96, 128)
544/700 [======================>.......] - ETA: 19:40 - loss: 0.2339 - accuracy: 0.1834 - mse: 0.0046(768, 1024, 3)
(96, 128)
545/700 [======================>.......] - ETA: 19:32 - loss: 0.2335 - accuracy: 0.1831 - mse: 0.0046(768, 1024, 3)
(9

603/700 [========================>.....] - ETA: 12:09 - loss: 0.2314 - accuracy: 0.1796 - mse: 0.0046(768, 1024, 3)
(96, 128)
604/700 [========================>.....] - ETA: 12:01 - loss: 0.2312 - accuracy: 0.1795 - mse: 0.0046(768, 1024, 3)
(96, 128)
605/700 [========================>.....] - ETA: 11:54 - loss: 0.2311 - accuracy: 0.1802 - mse: 0.0046(768, 1024, 3)
(96, 128)
606/700 [========================>.....] - ETA: 11:46 - loss: 0.2315 - accuracy: 0.1804 - mse: 0.0046(768, 1024, 3)
(96, 128)
607/700 [=========================>....] - ETA: 11:38 - loss: 0.2318 - accuracy: 0.1803 - mse: 0.0046(768, 1024, 3)
(96, 128)
608/700 [=========================>....] - ETA: 11:31 - loss: 0.2315 - accuracy: 0.1800 - mse: 0.0046(768, 1024, 3)
(96, 128)
609/700 [=========================>....] - ETA: 11:23 - loss: 0.2312 - accuracy: 0.1800 - mse: 0.0046(768, 1024, 3)
(96, 128)
610/700 [=========================>....] - ETA: 11:16 - loss: 0.2312 - accuracy: 0.1797 - mse: 0.0046(768, 1024, 3)
(9

668/700 [===========================>..] - ETA: 4:04 - loss: 0.2309 - accuracy: 0.1821 - mse: 0.0046(768, 1024, 3)
(96, 128)
669/700 [===========================>..] - ETA: 3:57 - loss: 0.2306 - accuracy: 0.1820 - mse: 0.0046(768, 1024, 3)
(96, 128)
670/700 [===========================>..] - ETA: 3:49 - loss: 0.2313 - accuracy: 0.1818 - mse: 0.0046(768, 1024, 3)
(96, 128)
671/700 [===========================>..] - ETA: 3:42 - loss: 0.2313 - accuracy: 0.1818 - mse: 0.0046(768, 1024, 3)
(96, 128)
672/700 [===========================>..] - ETA: 3:34 - loss: 0.2315 - accuracy: 0.1821 - mse: 0.0046(768, 1024, 3)
(96, 128)
673/700 [===========================>..] - ETA: 3:27 - loss: 0.2313 - accuracy: 0.1823 - mse: 0.0046(768, 1024, 3)
(96, 128)
674/700 [===========================>..] - ETA: 3:19 - loss: 0.2316 - accuracy: 0.1823 - mse: 0.0046(768, 1024, 3)
(96, 128)
675/700 [===========================>..] - ETA: 3:12 - loss: 0.2314 - accuracy: 0.1823 - mse: 0.0046(768, 1024, 3)
(96, 128)


In [24]:
model.save_weights('my_model_weight.h5')

In [25]:
save_mod(model,"my_model_weight.h5","Model.json")

In [26]:
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_B_test]